## Import and load the data file

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer

In [9]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kisha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kisha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

## Preprocess data

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [10]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


## Create training and testing data

In [14]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


## Build the model

In [16]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/500
47/47 [==============================] - 0s 2ms/step - loss: 2.2485 - accuracy: 0.1064
Epoch 2/500
47/47 [==============================] - 0s 297us/step - loss: 2.1508 - accuracy: 0.1702
Epoch 3/500
47/47 [==============================] - 0s 276us/step - loss: 2.1284 - accuracy: 0.2340
Epoch 4/500
47/47 [==============================] - 0s 301us/step - loss: 2.0275 - accuracy: 0.3191
Epoch 5/500
47/47 [==============================] - 0s 276us/step - loss: 1.9078 - accuracy: 0.4043
Epoch 6/500
47/47 [==============================] - 0s 275us/step - loss: 1.8635 - accuracy: 0.3830
Epoch 7/500
47/47 [==============================] - 0s 276us/step - loss: 1.7802 - accuracy: 0.4681
Epoch 8/500
47/47 [==============================] - 0s 276us/step - loss: 1.5095 - accuracy: 0.5532
Epoch 9/500
47/47 [==============================] - 0s 268us/step - loss: 1.3849 - accuracy: 0.7021
Epoch 10/500
47/47 [==============================] - 0s 276us/step - loss: 1.2933 - accuracy

47/47 [==============================] - 0s 212us/step - loss: 0.0223 - accuracy: 1.0000
Epoch 82/500
47/47 [==============================] - 0s 297us/step - loss: 0.0222 - accuracy: 1.0000
Epoch 83/500
47/47 [==============================] - 0s 233us/step - loss: 0.0356 - accuracy: 1.0000
Epoch 84/500
47/47 [==============================] - 0s 233us/step - loss: 0.0369 - accuracy: 1.0000
Epoch 85/500
47/47 [==============================] - 0s 276us/step - loss: 0.0958 - accuracy: 1.0000
Epoch 86/500
47/47 [==============================] - 0s 255us/step - loss: 0.0258 - accuracy: 1.0000
Epoch 87/500
47/47 [==============================] - 0s 191us/step - loss: 0.0177 - accuracy: 1.0000
Epoch 88/500
47/47 [==============================] - 0s 255us/step - loss: 0.0197 - accuracy: 1.0000
Epoch 89/500
47/47 [==============================] - 0s 254us/step - loss: 0.0259 - accuracy: 1.0000
Epoch 90/500
47/47 [==============================] - 0s 233us/step - loss: 0.0635 - accuracy: 

47/47 [==============================] - 0s 233us/step - loss: 0.0318 - accuracy: 1.0000
Epoch 161/500
47/47 [==============================] - 0s 254us/step - loss: 0.0516 - accuracy: 0.9787
Epoch 162/500
47/47 [==============================] - 0s 233us/step - loss: 0.0264 - accuracy: 0.9787
Epoch 163/500
47/47 [==============================] - 0s 233us/step - loss: 0.0162 - accuracy: 1.0000
Epoch 164/500
47/47 [==============================] - 0s 233us/step - loss: 0.0227 - accuracy: 1.0000
Epoch 165/500
47/47 [==============================] - 0s 233us/step - loss: 0.0160 - accuracy: 1.0000
Epoch 166/500
47/47 [==============================] - 0s 276us/step - loss: 0.0255 - accuracy: 1.0000
Epoch 167/500
47/47 [==============================] - 0s 255us/step - loss: 0.0250 - accuracy: 1.0000
Epoch 168/500
47/47 [==============================] - 0s 297us/step - loss: 0.0193 - accuracy: 1.0000
Epoch 169/500
47/47 [==============================] - 0s 233us/step - loss: 0.0224 - a

47/47 [==============================] - 0s 255us/step - loss: 0.0026 - accuracy: 1.0000
Epoch 240/500
47/47 [==============================] - 0s 297us/step - loss: 0.0052 - accuracy: 1.0000
Epoch 241/500
47/47 [==============================] - 0s 274us/step - loss: 0.0094 - accuracy: 1.0000
Epoch 242/500
47/47 [==============================] - 0s 276us/step - loss: 0.0225 - accuracy: 1.0000
Epoch 243/500
47/47 [==============================] - 0s 254us/step - loss: 0.0399 - accuracy: 0.9787
Epoch 244/500
47/47 [==============================] - 0s 233us/step - loss: 0.0043 - accuracy: 1.0000
Epoch 245/500
47/47 [==============================] - 0s 233us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 246/500
47/47 [==============================] - 0s 276us/step - loss: 0.0273 - accuracy: 1.0000
Epoch 247/500
47/47 [==============================] - 0s 276us/step - loss: 0.0039 - accuracy: 1.0000
Epoch 248/500
47/47 [==============================] - 0s 319us/step - loss: 5.8276e-04

47/47 [==============================] - 0s 276us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 319/500
47/47 [==============================] - 0s 297us/step - loss: 0.0028 - accuracy: 1.0000
Epoch 320/500
47/47 [==============================] - 0s 234us/step - loss: 0.0243 - accuracy: 1.0000
Epoch 321/500
47/47 [==============================] - 0s 232us/step - loss: 0.0042 - accuracy: 1.0000
Epoch 322/500
47/47 [==============================] - 0s 277us/step - loss: 0.0024 - accuracy: 1.0000
Epoch 323/500
47/47 [==============================] - 0s 297us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 324/500
47/47 [==============================] - 0s 254us/step - loss: 0.0071 - accuracy: 1.0000
Epoch 325/500
47/47 [==============================] - 0s 244us/step - loss: 0.0112 - accuracy: 1.0000
Epoch 326/500
47/47 [==============================] - 0s 233us/step - loss: 0.0029 - accuracy: 1.0000
Epoch 327/500
47/47 [==============================] - 0s 276us/step - loss: 0.0050 - a

47/47 [==============================] - 0s 233us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 398/500
47/47 [==============================] - 0s 212us/step - loss: 0.0025 - accuracy: 1.0000
Epoch 399/500
47/47 [==============================] - 0s 254us/step - loss: 0.0034 - accuracy: 1.0000
Epoch 400/500
47/47 [==============================] - 0s 255us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 401/500
47/47 [==============================] - 0s 255us/step - loss: 0.0231 - accuracy: 0.9787
Epoch 402/500
47/47 [==============================] - 0s 299us/step - loss: 0.0026 - accuracy: 1.0000
Epoch 403/500
47/47 [==============================] - 0s 212us/step - loss: 0.0028 - accuracy: 1.0000
Epoch 404/500
47/47 [==============================] - 0s 232us/step - loss: 0.0034 - accuracy: 1.0000
Epoch 405/500
47/47 [==============================] - 0s 276us/step - loss: 0.0483 - accuracy: 0.9787
Epoch 406/500
47/47 [==============================] - 0s 276us/step - loss: 0.0045 - a

47/47 [==============================] - 0s 276us/step - loss: 0.0159 - accuracy: 1.0000
Epoch 477/500
47/47 [==============================] - 0s 251us/step - loss: 0.0080 - accuracy: 1.0000
Epoch 478/500
47/47 [==============================] - 0s 259us/step - loss: 0.0025 - accuracy: 1.0000
Epoch 479/500
47/47 [==============================] - 0s 297us/step - loss: 0.0021 - accuracy: 1.0000
Epoch 480/500
47/47 [==============================] - 0s 233us/step - loss: 0.0038 - accuracy: 1.0000
Epoch 481/500
47/47 [==============================] - 0s 266us/step - loss: 0.0017 - accuracy: 1.0000
Epoch 482/500
47/47 [==============================] - 0s 275us/step - loss: 0.0192 - accuracy: 1.0000
Epoch 483/500
47/47 [==============================] - 0s 275us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 484/500
47/47 [==============================] - 0s 271us/step - loss: 0.0052 - accuracy: 1.0000
Epoch 485/500
47/47 [==============================] - 0s 297us/step - loss: 0.0025 - a